<a href="https://colab.research.google.com/github/talhavawda/tic-tac-toe-monte-carlo/blob/master/TicTacToe_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Tac Toe using Monte Carlo Tree Search

 ## COMP703 (ARTIFICIAL INTELLIGENCE) 2020 ASSIGNMENT 1
 
 Team: A1GroupWC

 Team Members:
 -  Azhar Mohamed&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218006491)
 - Yashlin Naidu &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(216019492)
 - Ricardo Pillay &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218009114)
 - Ahmad Jawaad Shah&ensp;&ensp;&ensp;(218029400)
 - Talha Vawda &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218023210)

<br>

## Acknowledgements and References



# Introduction and Background
Talk about game variant 

# Imports and constants
This cell contains all the libraries that is needed to run this game. Furthermore, constant initializations are done here and will be used throughout the notebook


In [12]:
"""
Imports
"""
import numpy as np
import math
import copy
import random
 
 
"""
Constants
"""
# 'C' is used in the UCT formula
# Theoretically, C should be set to sqrt(2) but play around with different values to see which works best
C = 2


class TicTacToeGame:
    """
        A class that represents our variant of the Tic Tac Toe game
        The purpose is to initialise a Tic Tac Toe game by creating a nxn square board
        and placing the neutral tokens on it, and to hold the n, k, and neutral_tokens values

        The board is being implemented as a nxn array with an index pair representing
        a square on the board and the element values being either [Deprecated: 'S', 'X', 'O', ""]
        2, 1, -1, 0 to represent a neutral token, the X token, the O token, and an empty square (where
        no token has been played yet)

        We are having a specific game class as we are demonstrating various examples of Tic Tac Toe game scenarios
        where different games could have a different size board, so we're going to need different game instances
        representing different games with their boards
    """

    # Constants of this Tic Tac Toe Game class
    X = 1 # Player/Token X
    EMPTY = 0 # An empty square (no tokens placed on it yet)
    O = -1 # Player/Token O
    S = 2 # Neutral Token
    IN_PROGRESS = 3 # Game in progress
    DRAW = 4 # Game over and game ended in draw


    def __init__(self, n):
        """
            :param n: Dimensions of the board - Since the board is a square board, the board will be an nxn board

            k is the number of tokens needed in a row to win
            neutral_tokens is the number of neutral tokens ('S') that the board will contain
        """
        self.n = n

        if self.n == 4:
            self.k = 3
            self.neutral_tokens = 1
        elif self.n in [5, 6]:
            self.k = 4
            self.neutral_tokens = 2
        elif self.n == 7:
            self.k = 5
            self.neutral_tokens = 3
        else:
            pass # We've already covered all cases and validation of n will be done when getting input from the user

  
    def initialise_game_board(self): 
        """
            Generate an initial board for the game with the neutral tokens (randomly) placed
             on it, and return it

            [Initially this was called insert_neutral_tokens()]

            :return: An initial board of the game (the initial game state)
        """

        # TO IMPLEMENT - ensure no neutral tokens are on the same row, column, or diagonal as any other neutral tokens
        
        board = [[self.EMPTY for i in range(self.n)] for j in range(self.n)] # initialise a nxn board using Python List Comprehension

        token = 0

        while token < self.neutral_tokens:
            randRow = random.randint(0, self.n-1) # For randint(a, b) both bounds are inclusive i.e. range is [a, b]
            randCol = random.randint(0, self.n-1)

            if board[randRow][randCol] != self.S: # ALT: board[randRow][randCol] == self.EMPTY
                board[randRow][randCol] = self.S
                token = token + 1

        return board
        


    def create_game_state(self, neutral_moves=None, players_moves=None):
        """
            Create a game situation based on the actions given

            :neutral_moves: a list of the squares on the board that the neutral tokens are placed on - each element
             is a tuple of size 2 indicating the position of the square.  the length of this list must be equal to self.neutral_tokens.
            If None then call self.initialise_game_board() to randomly place the neutral tokens
            :players_moves: moves that have already been played by both players - a list of playerToken-xyPosition pairs - ensure the positions are valid board positions
            :return: a State of the game populated with the specified actions
        """
        
        turn = self.X

        # TO IMPLEMENT
        if neutral_moves == None:
            board = self.initialise_game_board()
        else:
            board = [[self.EMPTY for i in range(self.n)] for j in range(self.n)] # initialise a nxn board using Python List Comprehension
            # Todo - add neutral moves

        if player_moves != None:

            #...(todo)

            """code to determine the player whose turn it is to play next on this current state"""
            num_x = (np.array(board) == self.X).sum()
            num_y = (np.array(board) == self.O).sum()

            if num_x > num_y:
                turn = self.O
            else:
                turn = self.X

        
        return State(board, turn)






# State Class

Represents a State of the Tic Tac Toe game

### Ideas
Rename this class to State and have a board variable

Have a field *player* that stores the current player whose turn it is

Functions to implement:
- __ init__()
- gameOver() # or isTerminal()
- getWinner()
- getBlankCells()
- getChildStates() - this is basically [for each action: result(player,action) -> append to list]
- utility(player) # evaluation of the current board state (or maybe only for terminal states) for the current player/token [we dont use a heuristic evaluation for MCTS]
- actions(player) # all the actions this player can do based on this current state  
- result(player, action) # new state to move to based on (this current state,) this action of this player


In [ ]:
class State:
    """
        A Tic Tac Toe state for the Monte Carlo Tree Search Algorithm
    """


    def __init__(self, board=None, turn=None):
        """
            Constructor for the initial creation of a Tic Tac Toe State.
            
            This constructor also checks the board to see if the game is over and updates
            the is_terminal_state and winner fields/variables of this class.
            These variables (instead of a function) will be used by other functions inside this class and also outside of this class
            (thus the check game over of this state process will only be done once, making the program more efficient)
        """

        if board != None:
            # Create a board for an existing game state
            self.board = board
        else:
            # Create the board for the initial game state
            self.board = TIC_TAC_TOE_GAME.initialise_game_board()
            self.turn = TIC_TAC_TOE_GAME.X # X plays first

        self.is_terminal_state = False
        self.winner = TIC_TAC_TOE_GAME.IN_PROGRESS 

        self.check_game_over() # This function updates self.is_terminal_state and self.winner 



        if turn != None:
            self.turn = turn # The player whose turn it is to currently move on this current state, either [Deprecated: "X" or "O"] TIC_TAC_TOE_GAME.X or TIC_TAC_TOE_GAME.O
        else:
            #Todo - determine whose turn it is by counting the number of X's and O's on the board. if X's==O's then X's turn otherwise O's
            num_x = (np.array(self.board) == TIC_TAC_TOE_GAME.X).sum()
            num_y = (np.array(self.board) == TIC_TAC_TOE_GAME.O).sum()
            if num_x > num_y:
              self.turn = TIC_TAC_TOE_GAME.O
            else:
              self.turn = TIC_TAC_TOE_GAME.X


    def get_child_states(self):
        """
            Generate and return all child states of this state with the current player being self.turn 

            Returns:
            possibleMoves: List States

            Use this method for expansion
        """
        state_possible_moves = list()
        board_possible_moves = list()
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if (self.board[i][j] == TIC_TAC_TOE_GAME.EMPTY):
                    temp_board = copy.deepcopy(self.board)
                    temp_board[i][j] = token
                    state_possible_moves.append(State(board=temp_board, turn=-self.turn))
                    board_possible_moves.append(temp_board)
        return state_possible_moves, board_possible_moves


    def moves(self):
        """
            Determine all the moves (x,y pairs) that can be played on the current board
            and return them

            :return: a list of tuples
        """
        # A list of tuples
        moves = list()
        for i in range(len(self.board)):
          for j in range(self.board[i]):
            # Check if the cell is empty
            if self.board[i][j] == TIC_TAC_TOE_GAME.EMPTY:
              # Create a tuple holding a possible i, j move
              move_tuple = (i, j)
              # Add the tuple to the list
              moves.append(move_tuple)
        return moves



    def play_move(self, move):
        """
            Play the specified move on this current state
            :return: a new State with the move having being played on it
        """
        # Assumption: move is a tuple
        # Play the move on this state
        new_board = copy.deepcopy(self.board)
        new_board[move[0]][move[1]] = self.turn

        # Return a new state - updated board - updated turn
        return State(board=new_board, turn=-self.turn) # turn=-self.turn since TIC_TAC_TOE_GAME.X == -TIC_TAC_TOE_GAME.O and vice versa


    def display_state(self):
        """
            A function for creating a display for the board
        """

        for i in range(len(self.board)):
            line = ""
            for j in range(len(self.board[i])):
                character = self.board[i][j]
                if character == TIC_TAC_TOE_GAME.EMPTY:
                    character = '_'
                elif character == TIC_TAC_TOE_GAME.X:
                    character = 'X'
                elif character == TIC_TAC_TOE_GAME.O:
                    character = 'O'
                else:
                    character ='S'
                line += "|" + str(character)
            line += "|"
            print(line)


    def toggle_turn(self, turn:int):
        """if turn == TIC_TAC_TOE_GAME.X:
            return TIC_TAC_TOE_GAME.O
        else:
            return TIC_TAC_TOE_GAME.X"""
        return -turn # since TIC_TAC_TOE_GAME.X == -TIC_TAC_TOE_GAME.O and vice versa
        

    def check_game_over(self):
        """
            Determines whether the game is over or not and updates the relevant fields/variables of this function
            The game is over if either X won, O won, or game is a draw
            If the game is over, then this state is a terminal state
        """

        # ***** TO IMPLEMENT *****
        # If the game is over, update the self.winner variable to either TIC_TAC_TOE_GAME.X, TIC_TAC_TOE_GAME.O, or TIC_TAC_TOE_GAME.DRAW and update the self.is_terminal_state to True
        # to check if there's no empty square left (for a draw condition), can use the following code: np.array(self.board == TIC_TAC_TOE_GAME.EMPTY)).sum() == 0


    def is_terminal(self):
        """
            return whether this state is a terminal state or not.
            This state is a terminal state if the game is over (checked by check_game_over() which updated the relevant variables) 
        """
        return self.is_terminal_state

    def get_winner(self):
        return self.winner

    def get_utilty(self):
        """
            Return the value of this (termnial state)
            Return 1 if X won, -1 if O won, 0.5 if there is a draw, or 0 if this state is not a terminal state
        """
        if self.is_terminal_state:
            if self.winner == TIC_TAC_TOE_GAME.DRAW:
                return 0.5
            else:
                return self.winner # TIC_TAC_TOE_GAME.X is represented by 1 and TIC_TAC_TOE_GAME.O is represented by -1 so the corresponding value for the winner will be returned
        else:
            return 0




    def get_empty_squares(self):
        """
            :return: a list of tuples, with each tuple representing the coordinates of an empty square on the board
        """
        empty_squares = list()
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
              if self.board[i][j] == TIC_TAC_TOE_GAME.EMPTY:
                  empty_square = tuple(i, j)
                empty_squares.append(empty_square)
        return empty_squares




# Node Class

[The root node of the tree is the current game state]

Todo:
Modify node/game class for adversarial game: 

Option 1 (this is what sir mentions in notes): have 2 separate win counters, one for player 1 (Max) and one for player 2 (Min) and the one that is used to calculate UCT value is the player who's turn it is to currently play (Me: in state 0 - the initial state). So if terminal value is positive then increment Player 1's score, else if negative then increment Player 2's score with the absolutve value of the terminal value. (We can also increment both players' scores (with half the value of a win - if win=1 then draw =0.5 for both players) if there is a draw). 
[AIMA mentions about having a vector of values per node for multiplayer games]

Option 2: have 1 score value, higher value indiciating preference to Max and lower value indicating preference to Min (thus value can go negative). So algo selects the node with either the best or worst UCT value depending on who's turn it is

Option 3 (See: https://en.wikipedia.org/wiki/Monte_Carlo_tree_search#Principle_of_operation): have 1 score/value counter for each node, and that score reperesents the player that that node represents. So when backpropagating, we update every alternate ascendant node with the score (the start of the alternation depends on whether the leaf node that was rolled out resulted in a win/loss for the player that that lead node represents). (So if selecting a child node, then the current node will have to choose the child with the minimum value as the values of the children represent that of the opposite player)
[This option is similar to Option 1]


Going with Option 1

Modelling: use MaxV and MinV for the scores of the 2 players in the Node representation. Max is the player whose turn it is to play in the initial state of the tree (the current state of the game)

In [ ]:
class Node:
  """
        A node for the Monte Carlo Tree Search algorithm, that is based on 
        a Tic Tac Toe State
  """
 
 
  def __init__(self, state: State, parent=None):
    """
    Constructor for the Node class

    Parameters:
    state (State): An instance of the state of the game
    parent (Node): The parent node of the current node
    """
    # The number of wins through this node
    self.v = 0
    # The number of times the node has been rolled out
    self.n = 0
    # The parent Node
    self.parent = parent
    # The Game State
    self.state = state
    # Chilren Nodes
    self.children = list()


  def add_child_nodes(self, token):
    """
    Method to create children of this node

    Returns:
    children: list of Nodes
    """
    # use this nodes board and get all possible boards.
    # Create a Node for each possibility
    actions = self.state.get_child_states(token) 
    for action in actions:
      self.children.append(Node(action, parent=self))
    pass # return children
 
 
  def calculate_uct_value(self):
    """
    A method to calculate the UCT value for a node
    
    Returns:
    uct: Float
    """
    # Check for 0 division
    if self.n == 0:
      # Return positive infinity (This node must be set to a high priority for Exploration)
      return np.inf
    else:
      # Return uct value for this node
      return (self.v/self.n) + C * math.sqrt(np.log(self.parent.n)/self.n)


  def select_child_node(self):
    """
    For the given node, identify which is the best child to be selected

    Returns:
    best_child: Node
    """
    max_uct = 0
    best_child = None
    for child in self.children:
      uct_value = child.calculate_uct_value()
      if uct_value > max_uct:
        max_uct = uct_value
        best_child = child
    return best_child


  def to_string(self):
    output = "Node Summary\nNumber of wins through this node: {}\nNumber of visits: {}\nUCT value = {}".format(self.v, self.n, self.calculate_uct_value())
    return output

# Monte Carlo Tree Search

AIMA: When the iterations terminate, the move with the highest number of Playouts/Rollouts is returned. You might think that it would be better to return the node with the highest average utility, but the idea is that a node with 65/100 wins is better than one with 2/3 wins, because the latter has a lot of uncertainty. In any event, the UCB1 formula ensures that the node with the most playouts is almost always the node with the highest win percentage, because the selection process favors win percentage more and more as the number of playouts goes up.


Todo - what happens if a node we at represents a terminal state and the algorithm wants to generate its child nodes


In [ ]:
def monte_carlo_tree_search(self, state):
    """
        Conduct the MCTS from the given state
    """
    pass

def random_action(state):
    """
        A function to get one random action 

        Returns:
        random_move: A tuple (i, j) to place a move
    """

    possible_moves = state.moves()
    option = random.randint(0, len(possible_moves)-1)
    random_move = possible_moves[option]
    return random_move


def simulate(state, move):
    """
        Apply a random action on the state

        Return:
        new_state: State
    """

    new_state = state.play_move(move)
    return new_state


def rollout(state):
  while True:
    if state.is_terminal():
        if state.check_win():
            return 1
        else:
            return 0.5
    action = random_action(state)
    state = simulate(state, action)


# TODO: Implement back propocation to parent nodes
def backpropogate(node, result):
    while node.parent != None:
        node.n += 1
        node.v += result
        node = node.parent


def monte_carlo_tree_search(node, iterations=100):
  """
  Conduct the MCTS from the given state
  """
  current = node
  count = 0
  
  while count < iterations:
    # is it a leaf
    if current.children==list():
        if current.state.check_win():
            return current
        if current.n == 0:
            result = rollout(node.state)
        else:
            current.add_child_nodes()
            current = node.select_child_node()
            result = rollout(current.state)
        backpropogate(current, result)
    else:
        current = current.select_child_node()
        current = current.select_child_node()


"""
Create an initial state
No X's or O's are present
"""
initial_state = State()
initial_state.display_state()

root = Node(initial_state)

print((np.array(initial_state.board)==2).sum())

complete_game_tree = monte_carlo_tree_search(node=root, iterations=200)
while not complete_game_tree.parent == None:
    complete_game_tree.state.display_state()




TODO: I didn't add Yashlins code to check if the S's are too close
|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|
|_|S|_|_|_|_|_|
|_|_|_|_|_|_|_|
|_|_|_|S|_|_|_|
|_|_|_|S|_|_|_|
3


'Monte Carlo Tree Search -  Rollout not done\ntoken = "X"\ncurrent = root\n# Is it a leaf node\nif current.children == list():\n  # Is n == 0\n  if current.n == 0:\n    # Conduct rollout\n    rollout(current.state.board, token)\n  else:\n    # Do expansion of the tree\n    current.add_child_nodes()\n    # Select best child\n    current = current.select_child_node()\n    # Conduct rollout\n    rollout(current.state.board, token)\nelse:\n  # Select best child\n  current = current.select_child_node()\n'

# Examples


In [ ]:

# Dimension of the board from user
try:
  size = int(input("Enter the size of the square board (4,5,6,7): "))

  if size < 4 or size > 7:
    size = 4 # Default if invalid value entered
except ValueError:
  size = 4 # Default if invalid (non-int) input

# Create game instance
TIC_TAC_TOE_GAME = TicTacToeGame(size)
initial_state = TIC_TAC_TOE_GAME


print("Board Size: {}x{}\nNumber of tokens needed to win: {}\nNumber of Neutral Tokens available: {}".format(TIC_TAC_TOE_GAME.n, TIC_TAC_TOE_GAME.n, TIC_TAC_TOE_GAME.k, TIC_TAC_TOE_GAME.neutral_tokens))